In [5]:
import json
from src.tracker import Tracker, TrackerVisualizer

import cv2
import numpy as np
import time

In [6]:
CONFIG_PATH = 'configs/test_set_2.json'

with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

print("Loaded config keys:", config.keys())
print("Colors available:", list(config.get("colors", {}).keys()))

Loaded config keys: dict_keys(['colors'])
Colors available: ['blue', 'red']


In [7]:
tracker = Tracker(config)
viz = TrackerVisualizer() 

In [8]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # si no es Windows: cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not access the webcam")

print("Webcam opened. Keys: [q]=quit | [m]=toggle masks")

show_masks = True
t0 = time.time()
frames_count = 0
fps = 0.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update tracker
    results = tracker.update(frame, return_masks=show_masks)

    # FPS
    frames_count += 1
    if frames_count % 10 == 0:
        t1 = time.time()
        fps = 10.0 / max(1e-6, (t1 - t0))
        t0 = t1

    # Draw (with visualizer)
    frame = viz.draw(frame, results)
    cv2.putText(
        frame,
        f"FPS: {fps:.1f}",
        (20, frame.shape[0] - 20),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 255),
        2
    )

    cv2.imshow("Tracker Test", frame)

    # Optional: show masks
    if show_masks and "_masks" in results:
        for cname, m in results["_masks"].items():
            cv2.imshow(f"mask_{cname}", m)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    elif key == ord("m"):
        show_masks = not show_masks
        if not show_masks:
            # close mask windows
            if "_masks" in results:
                for cname in results["_masks"].keys():
                    try:
                        cv2.destroyWindow(f"mask_{cname}")
                    except cv2.error:
                        pass

cap.release()
cv2.destroyAllWindows()
print("Done")


Webcam opened. Keys: [q]=quit | [m]=toggle masks
Done
